<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Kitti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pykitti

In [12]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q" -O kitti_dataset.zip

--2020-04-03 13:14:40--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%21100433&authkey=APrMUGQyaB4np4Q
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xwp44a.am.files.1drv.com/y4m6FRz82ytPJ25uxMA6Z8CBfOGSFzjMNj7MiJz_IpB00jLIjXzHHph3lGJAOk9u8m7kLLPD1Ct-i5CC8sAB1p75XQ4zxDpbaWmtWWFHx26ZFzuQzG7GCRGkbRV8kzaBE0llAm0HbqzgPUJPSliPfEl9bG4pZIisiHUnakfP3Mqt54d2I7jpxtiYo8Ei8XDLn-el-D1763hLSbiOlghqrnzEQ/kitti_dataset.zip?download&psid=1 [following]
--2020-04-03 13:14:41--  https://xwp44a.am.files.1drv.com/y4m6FRz82ytPJ25uxMA6Z8CBfOGSFzjMNj7MiJz_IpB00jLIjXzHHph3lGJAOk9u8m7kLLPD1Ct-i5CC8sAB1p75XQ4zxDpbaWmtWWFHx26ZFzuQzG7GCRGkbRV8kzaBE0llAm0HbqzgPUJPSliPfEl9bG4pZIisiHUnakfP3Mqt54d2I7jpxtiYo8Ei8XDLn-el-D1763hLSbiOlghqrnzEQ/kitti_dataset.zip?download&psid=1
Resolving xwp44a.am.files.1drv.com (xwp44a.am.fi

In [11]:
!unzip -qq kitti_dataset.zip -d data

replace data/dataset/sequences/04.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

import pykitti

basedir = '/content/data/dataset'

# Specify the dataset to load
sequence = '04'

# Load the data. Optionally, specify the frame range to load.
# dataset = pykitti.odometry(basedir, sequence)
dataset = pykitti.odometry(basedir, sequence)

In [0]:
def pcl_to_image(pointcloud, intrinsic_matrix, img_shape):
  """ Projects a pointcloud to camera image.

  Args:
      pointcloud (numpy array of shape (num_points, 6)):
          Point cloud in the format [x, y, z, r, g, b] for each point.
      camera_model (Instance of a camera model class): Camera model to use to
          reproject the point cloud on the virtual-camera image plane.
      image_width (int): Image width.
      image_height (int): Image height.

  Returns:
      depth_image (numpy array of shape (image_height, image_width),
          dtype=np.float32): Depth image (Z), with depth in m.
  """
  image_width = img_shape[1]
  image_height = img_shape[0]

  depth_image = np.zeros((image_height, image_width), dtype=np.float32)

  pcl_inside_view = pointcloud[pointcloud[:, 2] > 0, :]
  pcl_inside_view_xyz = np.hstack((pcl_inside_view[:, :3],
                                    np.ones((pcl_inside_view.shape[0], 1))))

  pcl_projected = np.array(intrinsic_matrix).dot(pcl_inside_view_xyz.T)
  pixel = np.rint(pcl_projected / pcl_projected[2, :]).astype(int)[:2, :]

  index_bool = np.logical_and(
      np.logical_and(0 <= pixel[0], pixel[0] < image_width),
      np.logical_and(0 <= pixel[1], pixel[1] < image_height))
  pixel = pixel[:, index_bool]

  pcl_inside_view = pcl_inside_view[index_bool, :]

  # Considering occlusion, we need to be careful with the order of assignment
  # descending order according to the z coordinate.
  index_sort = pcl_inside_view[:, 2].argsort()[::-1]
  pixel = pixel[:, index_sort]
  pcl_inside_view = pcl_inside_view[index_sort, :]

  depth_image[pixel[1], pixel[0]] = pcl_inside_view[:, 2]  # m to mm

  return depth_image

In [0]:
import numpy as np

# Grab some data
second_pose = dataset.poses[220]
first_rgb = dataset.get_rgb(219)
first_cam2 = dataset.get_cam2(221)
third_velo = dataset.get_velo(221)

P_cam2 = dataset.calib.P_rect_20
T_cam2_velo = dataset.calib.T_cam2_velo
import cv2

print(P_cam2.shape)
print(T_cam2_velo.shape)

cam2 = dataset.get_cam2(221)
cam3 = dataset.get_cam3(221)
cam2_g = cv2.cvtColor(np.asarray(cam2), cv2.COLOR_RGB2GRAY)
cam3_g = cv2.cvtColor(np.asarray(cam3), cv2.COLOR_RGB2GRAY)

stereo = cv2.StereoBM(numDisparities=16, blockSize=15)
print(stereo)
disparity = stereo.compute(cam2_g, cam3_g)

rgb_img = np.asarray(first_cam2)
print(rgb_img.shape)
depth_img = pcl_to_image(third_velo[:, :3], P_cam2.dot(T_cam2_velo), rgb_img.shape)

print(depth_img.shape)
cv2.imwrite('depth.png', depth_img * 1000)

# Display some of the data
np.set_printoptions(precision=4, suppress=True)
print('\nSequence: ' + str(dataset.sequence))
print('\nFrame range: ' + str(dataset.frames))

print('\nRGB stereo pair baseline [m]: ' + str(dataset.calib.b_rgb))

print('\nFirst timestamp: ' + str(dataset.timestamps[0]))
print('\nSecond ground truth pose:\n' + str(second_pose))

f, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].imshow(first_cam2)
ax[0].set_title('Left RGB Image (cam2)')

ax[1].imshow(depth_img * 1000)
ax[1].set_title('Right RGB Image (cam3)')

f2 = plt.figure()
ax2 = f2.add_subplot(111, projection='3d')
# Plot every 100th point so things don't get too bogged down
velo_range = range(0, third_velo.shape[0], 100)
ax2.scatter(third_velo[velo_range, 0],
            third_velo[velo_range, 1],
            third_velo[velo_range, 2],
            c=third_velo[velo_range, 3],
            cmap='gray')
ax2.set_title('Third Velodyne scan (subsampled)')

plt.show()